# "Modulo 5"
> "Analisi DMRG del modello di Heisenberg XXZ 1D"

In [117]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg as ssl
from numpy.linalg import multi_dot

## $$ H=-\sum [S_x S_x + S_y S_y +\delta S_z S_z]$$

In [320]:
Delta=0
m=10
NIter=10

# inizialize local ops

I=np.eye(2,2)
Sz=np.array([[0.5, 0],[0,-0.5]])
Sp=np.array([[0, 0],[1,0]])
Sm=np.array([[0, 1],[0,0]])

# initial blocks

BlockSz = Sz
BlockSp = Sp
BlockSm = Sm
BlockI  = I
BlockH  = np.zeros((2,2))
Energy = 0


for l in range(NIter):
    SystSize = 2*l + 4
    
# Get the 2m-dimensional operators for the block + site

    BlockH = np.kron(BlockH, I) - Delta * np.kron(BlockSz, Sz) + 0.5 * ( np.kron(BlockSp, Sm) + np.kron(BlockSm, Sp) )
    BlockSz = np.kron(BlockI, Sz)
    BlockSp = np.kron(BlockI, Sp)
    BlockSm = np.kron(BlockI, Sm)
    BlockI  = np.kron(BlockI, I)
    
# HAMILTONIAN MATRIX forsuperblock
    H_super = np.kron(BlockH, BlockI) + np.kron(BlockI, BlockH) - Delta * np.kron(BlockSz, BlockSz) + 0.5 * ( np.kron(BlockSp, BlockSm) + np.kron(BlockSm, BlockSp) )  
    H_super = 0.5 * (H_super + H_super.T);  # ensure H is symmetric
    
# Diagonalizing the Hamiltonian
    LastEnergy = Energy
    Energy= ssl.eigsh(H_super,1,which='SA')[0]#[0]
    Psi = ssl.eigsh(H_super,1,which='SA')[1].T#[0]
    EnergyPerBond = (Energy - LastEnergy) / 2
    Ener2 = Energy / SystSize
    
# Sigma = Psi' *kron(BlockSz,BlockSz) * Psi; % n.n. ZZ correlation function

# Form the reduced density matrix
    nr=Psi.size
    Dim = int(np.sqrt(nr))
    PsiMatrix = np.reshape(Psi,(Dim,Dim))
    Rho = np.dot(PsiMatrix, PsiMatrix.T)
    
# Diagonalize the density matrix
    D,V = np.linalg.eigh(Rho)
    #V=V.T
    D=D[::-1]  # descending
    Index=np.arange(Dim)
    Index=Index[::-1]
    V=V[:,Index]

# Construct the truncation operator
    NKeep = min(D.size, m)
    Omatr = V[:,:NKeep]
    TruncationError = 1 - sum(D[:NKeep])
    
# Transform the block operators into the truncated basis
    BlockH  = multi_dot([Omatr.T, BlockH, Omatr])
    BlockSz = multi_dot([Omatr.T, BlockSz, Omatr])
    BlockSp = multi_dot([Omatr.T, BlockSp, Omatr])
    BlockSm = multi_dot([Omatr.T, BlockSm, Omatr])
    BlockI =  multi_dot([Omatr.T, BlockI, Omatr])
    
    print(SystSize, Energy, EnergyPerBond, Ener2, TruncationError)
    
#Spectrum=ssl.eigsh(H_super)[0]

4 [-1.11803399] [-0.55901699] [-0.2795085] 0.0
6 [-1.7469796] [-0.31447281] [-0.29116327] 4.440892098500626e-16
8 [-2.37938524] [-0.31620282] [-0.29742316] 5.719729767594828e-07
10 [-3.01332844] [-0.3169716] [-0.30133284] 7.165868320946345e-07
12 [-3.64809254] [-0.31738205] [-0.30400771] 3.7881702987796473e-06
14 [-4.28332063] [-0.31761404] [-0.30595147] 4.350085748616728e-06
16 [-4.91886385] [-0.31777161] [-0.30742899] 9.750853244172575e-06
18 [-5.5545781] [-0.31785712] [-0.30858767] 1.0145299359498594e-05
20 [-6.19045287] [-0.31793739] [-0.30952264] 1.7108358645989696e-05
22 [-6.82638355] [-0.31796534] [-0.31029016] 1.6440535244632848e-05
